## Backtesting trading strategies

In [24]:
import pandas as pd 
import yfinance as yf
import sys
sys.path.append('../src')
from data import get_stock_data
from hydra import initialize, compose

In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
with initialize(version_base=None, config_path="../conf"):
    datapipeline_cfg = compose(config_name='datapipeline')
stock_data = get_stock_data(datapipeline_cfg)

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AHC: No timezone found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ADCT: Data doesn't exist for startDate = 1151640000, endDate = 1277870400
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- IMPX: No timezone found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AESC: Data doesn't exist for startDate = 1151640000, endDate = 1277870400
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- MITT: Data doesn't exist for startDate = 1151640000, endDate = 1277870400
[********************

KeyboardInterrupt: 

In [5]:
aapl = yf.Ticker("aapl")
aapl.dividends

Date
1987-05-11 00:00:00-04:00    0.000536
1987-08-10 00:00:00-04:00    0.000536
1987-11-17 00:00:00-05:00    0.000714
1988-02-12 00:00:00-05:00    0.000714
1988-05-16 00:00:00-04:00    0.000714
                               ...   
2022-02-04 00:00:00-05:00    0.220000
2022-05-06 00:00:00-04:00    0.230000
2022-08-05 00:00:00-04:00    0.230000
2022-11-04 00:00:00-04:00    0.230000
2023-02-10 00:00:00-05:00    0.230000
Name: Dividends, Length: 78, dtype: float64

In [7]:
data = pd.DataFrame()
for i in ['AMZN']:
    stock = yf.download(i, start="2017-01-01",
                   end="2017-04-30", interval = "1d",
                   )
    stock = stock.dropna().reset_index() 
    #stock = pd.MultiIndex.from_frame(stock,names=[])
stock

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-01-03,37.896000,37.938000,37.384998,37.683498,37.683498,70422000
1,2017-01-04,37.919498,37.984001,37.709999,37.859001,37.859001,50210000
2,2017-01-05,38.077499,39.119999,38.013000,39.022499,39.022499,116602000
3,2017-01-06,39.118000,39.972000,38.924000,39.799500,39.799500,119724000
4,2017-01-09,39.900002,40.088501,39.588501,39.846001,39.846001,68922000
...,...,...,...,...,...,...,...
76,2017-04-24,45.433998,45.499500,45.191002,45.370499,45.370499,62458000
77,2017-04-25,45.352001,45.473999,45.150002,45.381001,45.381001,67612000
78,2017-04-26,45.514999,45.787498,45.377998,45.464500,45.464500,52178000
79,2017-04-27,45.719501,46.092999,45.605499,45.918999,45.918999,106110000
